# plot EEGNet architecture and details

In [3]:
import mne
import sys, os
import numpy as np
import itertools
import pandas as pd

try:
    from braindecode.preprocessing import exponential_moving_standardize
    from braindecode import EEGClassifier
    from braindecode.util import set_random_seeds
except:
    from braindecode.preprocessing import exponential_moving_standardize
    from braindecode import EEGClassifier
    from braindecode.util import set_random_seeds
    
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from skorch.dataset import ValidSplit
from skorch.callbacks import LRScheduler
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict, cross_validate, StratifiedKFold
from sklearn.utils import compute_class_weight
import torch
torch.set_num_threads(1) # check if this helps with overload


/u/kroma/conda-envs/m4d/lib/python3.11/site-packages/docstring_inheritance/docstring_inheritors/bases/inheritor.py:329: DocstringInheritanceWarning: File /u/kroma/conda-envs/m4d/lib/python3.11/site-packages/braindecode/models/eegconformer.py:88: in EEGConformer.__init__: section Parameters: the docstring for the argument 'n_outputs' is missing.
  self._warn(
/u/kroma/conda-envs/m4d/lib/python3.11/site-packages/docstring_inheritance/docstring_inheritors/bases/inheritor.py:329: DocstringInheritanceWarning: File /u/kroma/conda-envs/m4d/lib/python3.11/site-packages/braindecode/models/eegconformer.py:88: in EEGConformer.__init__: section Parameters: the docstring for the argument 'n_chans' is missing.
  self._warn(
/u/kroma/conda-envs/m4d/lib/python3.11/site-packages/docstring_inheritance/docstring_inheritors/bases/inheritor.py:329: DocstringInheritanceWarning: File /u/kroma/conda-envs/m4d/lib/python3.11/site-packages/braindecode/models/eegconformer.py:88: in EEGConformer.__init__: section 

In [ ]:
# train model
net = EEGClassifier(
    "EEGNetv4", 
    #criterion__weight=torch.Tensor(class_weights).to('cpu'), # class weight
    module__final_conv_length='auto',
    train_split=None, #ValidSplit(0.2),
    max_epochs=200, 
    batch_size=16, 
    module__sfreq=256, 
    #optimizer__lr=lr,
    #module__drop_prob=0.25,
)


In [11]:
net.module__final_conv_length


'auto'

In [10]:
from braindecode.models import EEGNetv4

model = EEGNetv4(
    n_chans=30,
    n_times=308,
    n_outputs=2,
    final_conv_length='auto',
)
print(model)

Layer (type (var_name):depth-idx)                  Input Shape               Output Shape              Param #                   Kernel Shape
EEGNetv4 (EEGNetv4)                                [1, 30, 308]              [1, 2]                    --                        --
├─Ensure4d (ensuredims): 1-1                       [1, 30, 308]              [1, 30, 308, 1]           --                        --
├─Rearrange (dimshuffle): 1-2                      [1, 30, 308, 1]           [1, 1, 30, 308]           --                        --
├─Conv2d (conv_temporal): 1-3                      [1, 1, 30, 308]           [1, 8, 30, 309]           512                       [1, 64]
├─BatchNorm2d (bnorm_temporal): 1-4                [1, 8, 30, 309]           [1, 8, 30, 309]           16                        --
├─Conv2dWithConstraint (conv_spatial): 1-5         [1, 8, 30, 309]           [1, 16, 1, 309]           480                       [30, 1]
├─BatchNorm2d (bnorm_1): 1-6                       [1, 1

In [4]:
epochs = mne.read_epochs('/ptmp/kroma/m4d/data/processed/N170/sub-001/ica_ica_None_None_Cz_None_400ms_int-epo.fif')

Reading /ptmp/kroma/m4d/data/processed/N170/sub-001/ica_ica_None_None_Cz_None_400ms_int-epo.fif ...
    Found the data of interest:
        t =    -398.44 ...     800.78 ms
        0 CTF compensation matrices available
Adding metadata with 10 columns
160 matching events found
No baseline correction applied
0 projection items activated


In [8]:
epochs.info 

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,33 points
Good channels,30 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,256.00 Hz
Highpass,0.00 Hz
Lowpass,128.00 Hz


In [9]:
len(epochs.times)

308

# Time-resolved classifier structure

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from mne.decoding import (
    CSP,
    GeneralizingEstimator,
    LinearModel,
    Scaler,
    SlidingEstimator,
    Vectorizer,
    cross_val_multiscore,
    get_coef,
)

In [5]:
epochs

Number of events,160
Events,cars: 80faces: 80
Time range,-0.398 – 0.801 s
Baseline,-0.400 – 0.000 s


/tmp/ipykernel_11740/1192478647.py:1: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X = epochs.copy().get_data()


array([0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 1], dtype=int32)

In [8]:
X = epochs.copy().get_data()
y = epochs.events[:,-1] - 1 # subtract 1 to get 0 and 1 instead of 1 and 2

clf = make_pipeline(StandardScaler(), 
                    LogisticRegression(solver="liblinear", class_weight='balanced'))
time_decod = SlidingEstimator(clf, n_jobs=1, scoring="balanced_accuracy", verbose=True) 
scores = cross_val_multiscore(time_decod, X, y, cv=10, n_jobs=1) # new, n_jobs=1 to prevent overload


/tmp/ipykernel_11740/1088278260.py:1: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X = epochs.copy().get_data()
/u/kroma/conda-envs/m4d/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| Fitting SlidingEstimator : 308/308 [00:01<00:00,  276.75it/s]
100%|██████████| Fitting SlidingEstimator : 308/308 [00:00<00:00,  346.87it/s]
100%|██████████| Fitting SlidingEstimator : 308/308 [00:00<00:00,  363.02it/s]
100%|██████████| Fitting SlidingEstimator : 308/308 [00:00<00:00,  407.06it/s]
100%|██████████| Fitting SlidingEstimator : 308/308 [00:00<00:00,  363.39it/s]
100%|██████████| Fitting SlidingEstimator : 308/308 [00:00<00:00,  401.61it/s]
100%|██████████| Fitting SlidingEstimator : 308/308 [00:00<00:

In [9]:
time_decod

<SlidingEstimator({'allow_2d': False,
 'base_estimator': Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression',
                 LogisticRegression(class_weight='balanced',
                                    solver='liblinear'))]),
 'n_jobs': 1,
 'position': 0,
 'scoring': 'balanced_accuracy',
 'verbose': True})>

In [10]:
SlidingEstimator?

Init signature:
SlidingEstimator(
    base_estimator,
    scoring=None,
    n_jobs=None,
    *,
    position=0,
    allow_2d=False,
    verbose=None,
)
Docstring:     
Search Light.

Fit, predict and score a series of models to each subset of the dataset
along the last dimension. Each entry in the last dimension is referred
to as a task.

Parameters
----------

base_estimator : object
    The base estimator to iteratively fit on a subset of the dataset.

scoring : callable | str | None
    Score function (or loss function) with signature
    ``score_func(y, y_pred, **kwargs)``.
    Note that the "predict" method is automatically identified if scoring is
    a string (e.g. ``scoring='roc_auc'`` calls ``predict_proba``), but is
    **not**  automatically set if ``scoring`` is a callable (e.g.
    ``scoring=sklearn.metrics.roc_auc_score``).
n_jobs : int | None
    The number of jobs to run in parallel. If ``-1``, it is set
    to the number of CPU cores. Requires the :mod:`joblib` package

In [11]:
clf

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression',
                 LogisticRegression(class_weight='balanced',
                                    solver='liblinear'))])

In [12]:
LogisticRegression?

Init signature:
LogisticRegression(
    penalty='l2',
    *,
    dual=False,
    tol=0.0001,
    C=1.0,
    fit_intercept=True,
    intercept_scaling=1,
    class_weight=None,
    random_state=None,
    solver='lbfgs',
    max_iter=100,
    multi_class='auto',
    verbose=0,
    warm_start=False,
    n_jobs=None,
    l1_ratio=None,
)
Docstring:     
Logistic Regression (aka logit, MaxEnt) classifier.

In the multiclass case, the training algorithm uses the one-vs-rest (OvR)
scheme if the 'multi_class' option is set to 'ovr', and uses the
cross-entropy loss if the 'multi_class' option is set to 'multinomial'.
(Currently the 'multinomial' option is supported only by the 'lbfgs',
'sag', 'saga' and 'newton-cg' solvers.)

This class implements regularized logistic regression using the
'liblinear' library, 'newton-cg', 'sag', 'saga' and 'lbfgs' solvers. **Note
that regularization is applied by default**. It can handle both dense
and sparse input. Use C-ordered arrays or CSR matrices containi

In [14]:
clf.get_params()

{'memory': None,
 'steps': [('standardscaler', StandardScaler()),
  ('logisticregression',
   LogisticRegression(class_weight='balanced', solver='liblinear'))],
 'verbose': False,
 'standardscaler': StandardScaler(),
 'logisticregression': LogisticRegression(class_weight='balanced', solver='liblinear'),
 'standardscaler__copy': True,
 'standardscaler__with_mean': True,
 'standardscaler__with_std': True,
 'logisticregression__C': 1.0,
 'logisticregression__class_weight': 'balanced',
 'logisticregression__dual': False,
 'logisticregression__fit_intercept': True,
 'logisticregression__intercept_scaling': 1,
 'logisticregression__l1_ratio': None,
 'logisticregression__max_iter': 100,
 'logisticregression__multi_class': 'auto',
 'logisticregression__n_jobs': None,
 'logisticregression__penalty': 'l2',
 'logisticregression__random_state': None,
 'logisticregression__solver': 'liblinear',
 'logisticregression__tol': 0.0001,
 'logisticregression__verbose': 0,
 'logisticregression__warm_start':

In [15]:
SlidingEstimator?

Init signature:
SlidingEstimator(
    base_estimator,
    scoring=None,
    n_jobs=None,
    *,
    position=0,
    allow_2d=False,
    verbose=None,
)
Docstring:     
Search Light.

Fit, predict and score a series of models to each subset of the dataset
along the last dimension. Each entry in the last dimension is referred
to as a task.

Parameters
----------

base_estimator : object
    The base estimator to iteratively fit on a subset of the dataset.

scoring : callable | str | None
    Score function (or loss function) with signature
    ``score_func(y, y_pred, **kwargs)``.
    Note that the "predict" method is automatically identified if scoring is
    a string (e.g. ``scoring='roc_auc'`` calls ``predict_proba``), but is
    **not**  automatically set if ``scoring`` is a callable (e.g.
    ``scoring=sklearn.metrics.roc_auc_score``).
n_jobs : int | None
    The number of jobs to run in parallel. If ``-1``, it is set
    to the number of CPU cores. Requires the :mod:`joblib` package

In [16]:
cross_val_multiscore?

Signature:
cross_val_multiscore(
    estimator,
    X,
    y=None,
    groups=None,
    scoring=None,
    cv=None,
    n_jobs=None,
    verbose=None,
    fit_params=None,
    pre_dispatch='2*n_jobs',
)
Docstring:
Evaluate a score by cross-validation.

Parameters
----------
estimator : instance of sklearn.base.BaseEstimator
    The object to use to fit the data.
    Must implement the 'fit' method.
X : array-like, shape (n_samples, n_dimensional_features,)
    The data to fit. Can be, for example a list, or an array at least 2d.
y : array-like, shape (n_samples, n_targets,)
    The target variable to try to predict in the case of
    supervised learning.
groups : array-like, with shape (n_samples,)
    Group labels for the samples used while splitting the dataset into
    train/test set.
scoring : str, callable | None
    A string (see model evaluation documentation) or
    a scorer callable object / function with signature
    ``scorer(estimator, X, y)``.
    Note that when using an es